In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import LOCM3
from utils import read_plan, read_json_file
from evaluator import ExecutabilityEvaluator
from collections import defaultdict
from itertools import combinations, permutations

In [ ]:
def test_acceptance(domain_name,train_traces, test_traces, invalid_suffixes):
    debug = {}
    try:
        extractor = LOCM3( debug=debug)
        model = extractor.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)
        
        evaluator = ExecutabilityEvaluator(learned_domain, debug=False)
        valid_res = []
        invalid_res = []
        for problem, trace in test_traces.items():
            valid_acceptance, invalid_acceptance = evaluator.get_acceptance_rate(trace, invalid_suffixes[problem])
            valid_res.append(valid_acceptance)
            invalid_res.append(invalid_acceptance)
        if len(valid_res) == 0:
            valid = 0
        else:
            valid = sum(valid_res) / len(valid_res)
        if len(invalid_res) == 0:
            invalid = 0
        else:
            invalid = sum(invalid_res) / len(invalid_res)
        return valid, invalid
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,1


In [3]:
# test_plan = "(pickup b2?object),(stack b2?object b1?object),(unstack b2?object b1?object),(putdown b2?object),(pickup b3?object),(stack b3?object b2?object)"
# test_trace = read_plan(test_plan)


# # test_plan2 = "(pickup b2?object),(stack b2?object b1?object),(unstack b3?object b4?object),(putdown b3?object)"
# # test_trace2 = read_plan(test_plan)

# debug = {
#     'trace_to_obj_trace' : True
# }
# l3 = LOCM3(event_arity=2, debug=debug, viz=True)
# model = l3.extract_model([test_trace])


In [4]:
# pddl_domain = model.to_pddl_domain("test_domain")
# pddl_domain.dump()
# evaluator = ExecutabilityEvaluator(pddl_domain, debug=True)
# exe = evaluator.get_acceptance_rate(test_trace, None)
# print(exe)

In [5]:
plain_traces = defaultdict(lambda: defaultdict())
with open("../../data/plain_traces/plain_traces.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        details = line.split("&&")

        domain_name = details[0]
        problem_name = details[2]
        plan = details[-1]

        plain_traces[domain_name][problem_name]= read_plan(plan)



In [6]:
invalid_suffixes = defaultdict(lambda: defaultdict(list))

with open("../../data/plain_traces/invalid_suffixes.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        details = line.split("&&")
        domain_name = details[0]
        problem_name = details[1]
        plan = details[2]
        
        invalid_suffixes[domain_name][problem_name].append(read_plan(plan)[0])

In [7]:
train_traces = defaultdict(list)
data = read_json_file("../../data/training_data/traces_plan_po_r10.json")
for learning_obj in data:
    domain_name = learning_obj["domain"]
    train_traces[domain_name].append(learning_obj)

In [8]:
with open("./acceptance_rate.csv", "w") as f:
    f.write("ID, Domain, len, len%, acceptance_rate, invalid_acceptance_rate\n")
    for domain, items in train_traces.items():
        print(f"Testing domain: {domain}")
        for item in items:
            traces = item["traces"]
            problem_name
            try:
                rate, invalid_rate = test_acceptance(domain, traces, plain_traces[domain], invalid_suffixes[domain])
            except Exception as e:
                print(f"Error in balanced executability for domain {domain}: {e}")
                continue

        # Test balanced executability
            f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {rate}, {invalid_rate}\n")
            f.flush()
            print(f"Balanced Executability for {domain}: {(rate, invalid_rate)}")

Testing domain: blocksworld
Error in balanced executability for domain blocksworld: test_acceptance() takes 3 positional arguments but 4 were given
Error in balanced executability for domain blocksworld: test_acceptance() takes 3 positional arguments but 4 were given
Error in balanced executability for domain blocksworld: test_acceptance() takes 3 positional arguments but 4 were given
Error in balanced executability for domain blocksworld: test_acceptance() takes 3 positional arguments but 4 were given
Error in balanced executability for domain blocksworld: test_acceptance() takes 3 positional arguments but 4 were given
Error in balanced executability for domain blocksworld: test_acceptance() takes 3 positional arguments but 4 were given
Error in balanced executability for domain blocksworld: test_acceptance() takes 3 positional arguments but 4 were given
Error in balanced executability for domain blocksworld: test_acceptance() takes 3 positional arguments but 4 were given
Error in bal